In [0]:
%sql

CREATE OR REPLACE TABLE datamodeling.gold.dimshow
WITH rendup AS
(
SELECT
    DISTINCT(show_id),
    type,
    title,
    description,
    duration_minutes,
    duration_seasons
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY show_id) as DIMSHOWKEY
FROM rendup


In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dimdirector
WITH remdup AS
(
SELECT
    DISTINCT(director)
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY director) as DIMDIRECTORKEY
FROM remdup
   


In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dimcast
WITH remdup AS
(
SELECT
    DISTINCT(cast)
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY cast) as DIMCASTKEY
FROM remdup

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dim_added_date
WITH remdup AS
(
SELECT
    DISTINCT(date_added)
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY date_added) as DIMDATEKEY
FROM remdup

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dim_ratings
WITH remdup AS
(
SELECT
    DISTINCT(rating)
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY rating) as DIMRATEKEY
FROM remdup

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dimcountry
WITH remdup AS
(
SELECT
    DISTINCT(country)
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY country) as DIMCOUNTRYKEY
FROM remdup

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dimaddinfo
WITH remdup AS
(
SELECT
    DISTINCT(date_added_clean),
    ingestion_date,
    source_file
FROM datamodeling.silver.silver_source
)
SELECT *,
   row_number() OVER(ORDER BY date_added_clean) as DIMADDINFOKEY
FROM remdup

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.dimmovieinfo
SELECT 
row_number() OVER(ORDER BY show_id) as dimmoviekey,
show_id,
type,
title,
director,
cast,
country,
date_added,
rating,
duration,
listed_in,
description,
ingestion_date,
source_file,
duration_minutes,
duration_seasons,
date_added_clean
from datamodeling.silver.silver_source

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.fact
SELECT 
  S.release_year,
  MI.dimmoviekey,
  AI.DIMADDINFOKEY,
  R.DIMRATEKEY,
  C.DIMCOUNTRYKEY,
  DA.DIMDATEKEY,
  CA.DIMCASTKEY,
  DR.DIMDIRECTORKEY,
  D.DIMSHOWKEY
FROM datamodeling.silver.silver_source S
LEFT JOIN datamodeling.gold.dimshow D
ON D.show_id = S.show_id
LEFT JOIN datamodeling.gold.dimcountry C
ON C.country = S.country
LEFT JOIN datamodeling.gold.dim_ratings R
ON R.rating = S.rating
LEFT JOIN datamodeling.gold.dim_added_date DA
ON DA.date_added = S.date_added
LEFT JOIN datamodeling.gold.dimcast CA
ON CA.cast = S.cast
LEFT JOIN datamodeling.gold.dimdirector DR
ON DR.director = S.director
LEFT JOIN datamodeling.gold.dimaddinfo AI
ON AI.date_added_clean = S.date_added_clean
LEFT JOIN datamodeling.gold.dimmovieinfo MI
ON MI.show_id = S.show_id